In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Generalized_Anxiety_Disorder/GSE61672'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Blood gene expression profiles associated with symptoms of generalized anxiety disorder"
!Series_summary	"Prospective epidemiological studies found that generalized anxiety disorder (GAD) can impair immune function and increase risk for cardiovascular disease or events. Mechanisms underlying the physiological reververations of anxiety, however, are still elusive. Hence, we aimed to investigate molecular processes mediating effects of anxiety on physical health using blood gene expression profiles of 546 community participants. Of these, 179 met the status of controls and 157 cases of anxiety."
!Series_overall_design	"We examined genome-wide differential gene expression in anxiety, as well as associations between nine major modules of co-regulated transcripts in blood gene expression and anxiety. There were a total of 546 subjects."
Sample Characteristics Dictionary:
{0: ['age: 44', 'age: 59', 'age: 39', 'age: 64', 'age: 58', 'age: 45', 'age: 37', 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 4
age_row = 0
gender_row = 1

def convert_trait(value):
    try:
        value = value.split(":")[1].strip().lower()
        if "case" in value:
            return 1
        elif "control" in value:
            return 0
    except (AttributeError, IndexError):
        return None
    return None

def convert_age(value):
    try:
        return float(value.split(":")[1].strip())
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    try:
        if "f" in value.lower():
            return 0
        elif "m" in value.lower():
            return 1
    except AttributeError:
        return None
    return None

# Save cohort information
save_cohort_info('GSE61672', './preprocessed/Generalized_Anxiety_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    # Extract and save clinical features
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Generalized_Anxiety_Disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Generalized_Anxiety_Disorder/trait_data/GSE61672.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1510561': [nan, 44.0, 0.0], 'GSM1510562': [nan, 59.0, 0.0], 'GSM1510563': [nan, 44.0, 0.0], 'GSM1510564': [nan, 39.0, 0.0], 'GSM1510565': [nan, 64.0, 0.0], 'GSM1510566': [nan, 58.0, 1.0], 'GSM1510567': [nan, 45.0, 1.0], 'GSM1510568': [nan, 37.0, 1.0], 'GSM1510569': [nan, 40.0, 1.0], 'GSM1510570': [nan, 39.0, 0.0], 'GSM1510571': [nan, 57.0, 1.0], 'GSM1510572': [nan, 52.0, 0.0], 'GSM1510573': [nan, 59.0, 0.0], 'GSM1510574': [nan, 57.0, 1.0], 'GSM1510575': [nan, 62.0, 0.0], 'GSM1510576': [nan, 62.0, 1.0], 'GSM1510577': [nan, 55.0, 0.0], 'GSM1510578': [nan, 55.0, 0.0], 'GSM1510579': [nan, 53.0, 1.0], 'GSM1510580': [nan, 47.0, 1.0], 'GSM1510581': [nan, 48.0, 0.0], 'GSM1510582': [nan, 49.0, 0.0], 'GSM1510583': [nan, 35.0, 1.0], 'GSM1510584': [nan, 58.0, 1.0], 'GSM1510585': [nan, 46.0, 0.0], 'GSM1510586': [nan, 54.0, 0.0], 'GSM1510587': [nan, 67.0, 1.0], 'GSM1510588': [nan, 47.0, 0.0], 'GSM1510589': [nan, 51.0, 0.0], 'GSM1510590': [nan, 34.0, 1.0], 'GSM1510591': [nan, 58.0, 1.0], 'GSM151

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651254', 'ILMN_1651262', 'ILMN_1651278', 'ILMN_1651296',
       'ILMN_1651315', 'ILMN_1651316', 'ILMN_1651336', 'ILMN_1651346',
       'ILMN_1651347', 'ILMN_1651354', 'ILMN_1651358', 'ILMN_1651364',
       'ILMN_1651378', 'ILMN_1651385', 'ILMN_1651403', 'ILMN_1651405'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene data
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Generalized_Anxiety_Disorder/gene_data/GSE61672.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Generalized_Anxiety_Disorder')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE61672', './preprocessed/Generalized_Anxiety_Disorder/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Generalized_Anxiety_Disorder/GSE61672.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Generalized_Anxiety_Disorder', the least common label is '1.0' with 62 occurrences. This represents 43.97% of the dataset.
The distribution of the feature 'Generalized_Anxiety_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 41.0
  50% (Median): 49.0
  75%: 56.0
Min: 18.0
Max: 82.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 141 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.

A new JSON file was created at: ./preprocessed/Generalized_Anxiety_Disorder/cohort_info.json
